# Amide-aromatic interaction analysis using PDB and BMRB archives
## Introduction
The objective of the notebook is to understand the anomalous chemical shifts of amide protons as a result of hydrogen bond interaction between the amide proton  and the near by aromatic side-chain. For this study we are going to use the chemical shift information available in BMRB database (https://bmrb.io) and the geometric information available in wwPDB(http://www.wwpdb.org). 

### Quick overview of the steps

* Find the matching BMRB and PDB Ids. This information is available via BMRB-API(http://api.bmrb.io/v2/mappings/bmrb/pdb?match_type=exact) and as well as BMRB-FTP (https://bmrb.io/ftp/pub/bmrb/nmr_pdb_integrated_data/adit_nmr_matched_pdb_bmrb_entry_ids.csv)
* Extract the amide chemical shifts from the NMR-STAR file and calculate the Z-score
* Extract the coordinate information from CIF file and calculate the distance between the amide proton and the center of the nearest aromatic ring, azimuthal angle and solid angle for each amide proton. 
* Combine the information from NMR-STAR(BMRB) and CIF(wwPDB) using atom identifier(seq_id,chain_id,residue,atom)
* write the output as csv file
* repeat the step for all matching PDB and BMRB ids. 

## Import the necessary python packages 
Please make sure you have installed all the necessary packages(pynmrstar,mmcif and numpy). For plotting you may need plotly and csv

In [ ]:
import pynmrstar
from mmcif.io.PdbxReader import PdbxReader
import os
import numpy
import csv
import gzip
import plotly.express as px

##  Extract chemical shift information and calculate the Z-score

We are going to use the pynmrstr (https://pypi.org/project/pynmrstar/) package to parse the NMR-STAR file. The Z-score is calculated using mean and the standard deviation from the BMRB statistics (https://bmrb.io/ref_info/stats.php?restype=aa&set=filt)
Let us first write a function to calculate the Z-score
### Function to calculate Z-score

In [ ]:
def get_z_score(res, x):
    m = {'ALA':8.194,'ARG':8.234,'ASN':8.324,'ASP':8.300,'CYS':8.386,'GLN':8.219,'GLU':8.330,'GLY':8.330,
         'HIS':8.247,'ILE':8.262,'LEU':8.215,'LYS':8.177,'MET':8.251,'PHE':8.335,'SER':8.277,'THR':8.232,
         'TRP':8.264,'TYR':8.289,'VAL':8.270}
    sd = {'ALA':0.577,'ARG':0.601,'ASN':0.610,'ASP':0.558,'CYS':0.670,'GLN':0.569,'GLU':0.576,'GLY':0.619,
         'HIS':0.666,'ILE':0.674,'LEU':0.627,'LYS':0.589,'MET':0.575,'PHE':0.710,'SER':0.568,'THR':0.610,
         'TRP':0.761,'TYR':0.721,'VAL':0.659}
    try:
        sp = (x-m[res])/sd[res]
    except KeyError:
        sp = 0.00
    return round(sp,3)

The above function will calculate the Z-score for a given amide proton chemical shift and residue type. For example

In [ ]:
get_z_score('LEU',5.33)

## Function to parse the chemical shift and calculate the Z-score
First let us define aromatic residues and their atoms. 

In [ ]:
aromatic_atoms = {
    'PHE': ['CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ', 'HD1', 'HD2', 'HE1', 'HE2', 'HZ'],
    'TYR': ['CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ', 'HD1', 'HD2', 'HE1', 'HE2', 'HH'],
    'TRP': ['CD2', 'CE2', 'CE3', 'CZ2', 'CZ3', 'CH2', 'HE3', 'HZ2', 'HZ3', 'HH2', 'HE1'],
    'HIS': ['CG', 'ND1', 'CD2', 'CE1', 'NE2', 'HD1', 'HD2', 'HE1', 'HE2', 'xx', 'yy']  
}

We also extract the number of unique chains(entity) and the number of copies of the chain(assembly) from the NMR-STAR file. There are two kid of sequence numbering available in both BMRB and PDB. One is author provided and other one is BMRB/PDB annotated. We have the option to chose which sequence number we want to extract from the file. This is crucial in the later stage while we combining data from BMRB and PDB.

The following function will create chemical shift dictionary from the star file

In [ ]:
def get_chemical_shifts(str_file,auth_tag=False):
    str_data = pynmrstar.Entry.from_file(str_file)
    csdata = str_data.get_loops_by_category('Atom_chem_shift')
    entity = str_data.get_tag('_Entity_assembly.Entity_ID')
    entity_size = len(set(entity))
    assembly_size = (len(entity))
    amide_cs = {}
    aromatic_cs = {}
    for cs in csdata:
        tag_list = cs.get_tag_names()

        id2 = tag_list.index('_Atom_chem_shift.Auth_asym_ID')
        if auth_tag:
            id1 = tag_list.index('_Atom_chem_shift.Auth_seq_ID')
        else:
            id1 = tag_list.index('_Atom_chem_shift.Comp_index_ID')
        id3 = tag_list.index('_Atom_chem_shift.Comp_ID')
        id4 = tag_list.index('_Atom_chem_shift.Atom_ID')
        id5 = tag_list.index('_Atom_chem_shift.Val')
        id6 = tag_list.index('_Atom_chem_shift.Ambiguity_code')
        for d in cs.data:
            if d[id4] == 'H':
                if d[id2] == '.':
                    d[id2] = 'A'  # temp fix if asym id is missing
                amide_cs[(d[id1], d[id2], d[id3], d[id4])] = (d[id5],get_z_score(d[id3],float(d[id5])))
            if d[id3] in aromatic_atoms.keys():
                if d[id2] == '.':
                    d[id2] = 'A'  # temp fix if asym id is missing
                k = (d[id1], d[id2], d[id3])
                if d[id4] in aromatic_atoms[d[id3]]:
                    if k not in aromatic_cs.keys():
                        aromatic_cs[k] = {}
                    aromatic_cs[k][d[id4]] = (d[id5], d[id6])
    return amide_cs, aromatic_cs, entity_size, assembly_size

There are two ways one can get the required NMR-STAR file, either by directly downloading the file from BMRB or by using reboxitory in NMRBox. 

In [ ]:
def get_bmrb_data(bmrb_id,auth_tag = False, nmrbox=False):
    if nmrbox:
        str_file_path = '/reboxitory/2021/07/BMRB/macromolecules/bmr{}/bmr{}_3.str'.format(
            bmrb_id,bmrb_id
        )
        bmrb_data = get_chemical_shifts(str_file_path, auth_tag)
    else:
        if not os.path.isdir('./data'):
            os.system('mkdir ./data')
        if not os.path.isdir('./data/BMRB'):
            os.system('mkdir ./data/BMRB')
        str_file = './data/BMRB/{}.str'.format(bmrb_id)
        if not os.path.isfile(str_file):
            cmd = 'wget http://rest.bmrb.io/bmrb/{}/nmr-star3 -O ./data/BMRB/{}.str'.format(bmrb_id, bmrb_id)
            os.system(cmd)
        bmrb_data = get_chemical_shifts('./data/BMRB/{}.str'.format(bmrb_id),auth_tag)
    return bmrb_data

For example let us fetch the chemical shift information for the BMRB entry  11086

In [ ]:
bmrb_data = get_bmrb_data('11086')

In [ ]:
amide_cs,aromatic_cs,entity_size,assembly_size = bmrb_data

Now we have all the required information in a dictionary format

In [ ]:
amide_cs

In [ ]:
aromatic_cs

In [ ]:
entity_size

In [ ]:
assembly_size

## Extract coordinate data from CIF file
We use mmcif library to parse the CIF file. The following function will parse the CIF file and extract the coordinate information as a dictionary

In [ ]:

def get_coordinates(cif_file, use_auth_tag=True, gzfile = False):
    """
    Extract coordinate information from cif file as a dictionary
    {model_id : {(seq_id,chain_id,res_id,atom_id) : array[x,y,x],...},...}
    :param cif_file: Input coordinate file
    :return: dictionary
    """
    cif_data = []
    if gzfile:
        ifh = gzip.open(cif_file,'rt') # For NMRBox
    else:
        ifh = open(cif_file, 'r')
    pRd = PdbxReader(ifh)
    pRd.read(cif_data)
    ifh.close()
    c0 = cif_data[0]
    atom_site = c0.getObj('atom_site')
    max_models = int(atom_site.getValue('pdbx_PDB_model_num', -1))
    col_names = atom_site.getAttributeList()
    model_id = col_names.index('pdbx_PDB_model_num')
    x_id = col_names.index('Cartn_x')
    y_id = col_names.index('Cartn_y')
    z_id = col_names.index('Cartn_z')
    atom_id = col_names.index('label_atom_id')
    comp_id = col_names.index('label_comp_id')
    asym_id = col_names.index('label_asym_id')
    entity_id = col_names.index('label_entity_id')
    seq_id = col_names.index('label_seq_id')
    icode_id = col_names.index('pdbx_PDB_ins_code')
    alt_id = col_names.index('label_alt_id')
    aut_seq_id = col_names.index('auth_seq_id')
    aut_asym_id = col_names.index('auth_asym_id')
    aut_atom_id = col_names.index('auth_atom_id')
    aut_comp_id = col_names.index('auth_comp_id')
    pdb_models = {}
    atom_ids = {}
    for model in range(1, max_models + 1):
        pdb = {}
        aid = {}
        for dat in atom_site.getRowList():
            if dat[atom_id] == 'H' or dat[comp_id] in aromatic_atoms.keys():  # Only necessary coordinates for this
                # calculation
                if int(dat[model_id]) == model:
                    if use_auth_tag:
                        aid[(dat[aut_seq_id], dat[aut_asym_id], dat[aut_comp_id], dat[aut_atom_id])] = \
                            (dat[entity_id], dat[asym_id], dat[comp_id], dat[seq_id], dat[aut_seq_id],
                             dat[alt_id], dat[icode_id], dat[aut_asym_id])
                        pdb[(dat[aut_seq_id], dat[aut_asym_id], dat[aut_comp_id], dat[aut_atom_id])] = \
                            numpy.array([float(dat[x_id]), float(dat[y_id]), float(dat[z_id])])
                    else:
                        aid[(dat[seq_id], dat[asym_id], dat[comp_id], dat[atom_id])] = \
                            (dat[entity_id], dat[asym_id], dat[comp_id], dat[seq_id], dat[aut_seq_id],
                             dat[alt_id], dat[icode_id], dat[aut_asym_id])
                        pdb[(dat[seq_id], dat[asym_id], dat[comp_id], dat[atom_id])] = \
                            numpy.array([float(dat[x_id]), float(dat[y_id]), float(dat[z_id])])
        pdb_models[model] = pdb
        atom_ids[model] = aid
    return pdb_models


As before, we can either fetch the CIF file from wwPDB FTP archive or use the reboxitory in NMRBox

In [ ]:

def get_pdb_data(pdb_id, auth_tag=False, nmrbox=False):
    if nmrbox:
        cif_file_path = '/reboxitory/2021/07/PDB/data/structures/all/mmCIF/{}.cif.gz'.format(pdb_id.lower())
        pdb_data = get_coordinates(cif_file_path,auth_tag,gzfile=True)
    else:
        if not os.path.isdir('./data'):
            os.system('mkdir ./data')
        if not os.path.isdir('./data/PDB'):
            os.system('mkdir ./data/PDB')
        cif_file = './data/PDB/{}.cif'.format(pdb_id)
        if not os.path.isfile(cif_file):
            cmd = 'wget https://files.rcsb.org/download/{}.cif -O ./data/PDB/{}.cif'.format(pdb_id, pdb_id)
            os.system(cmd)
        pdb_data = get_coordinates('./data/PDB/{}.cif'.format(pdb_id),auth_tag)
    return pdb_data

For example let us extract the coordinate data for PDB 1WYO

In [ ]:
pdb_data = get_pdb_data('1WYO')

Now we have coordinate data in a dictionary format  {model_id:{atom_identifier:array(x,y,z)}}

In [ ]:
pdb_data

## Calculate the distance, azimuthal angle and solid angle

Before we begin, let us define some useful functions
### Distance between two points in space

In [ ]:

def get_distance(c1, c2):
    """
    Calculates the distance between two coordinate points
    :param c1: array of x,y,z
    :param c2: array of x,y,z
    :return: distance between two ponts
    """
    return numpy.linalg.norm(c1 - c2)

### Centroid of given set of points in a plane

In [ ]:
def get_centroid(p):
    # print (len(p),p)
    x = [i[0] for i in p]
    y = [i[1] for i in p]
    z = [i[2] for i in p]
    c = [sum(x) / len(p), sum(y) / len(p), sum(z) / len(p)]
    return numpy.array(c)


### Solid angle for a given azimuthal angle at the distance

In [ ]:
def solid_angle(a_deg, r):
    s = 1.4 #C-C bond length
    A=((3.0*numpy.sqrt(3))/2.0)*s*s #area of the hexagonal plane
    a = (numpy.pi / 180) * a_deg
    # sa2=2*numpy.pi*(1.0-1.0/(numpy.sqrt(1+(A*numpy.cos(a)/(numpy.pi*r1**r1)))))
    sa = 2 * numpy.pi * (1.0 - 1.0 / (numpy.sqrt(1 + (A*numpy.cos(a) / (numpy.pi * r * r)))))
    # print (a_deg)
    sa_deg = (180.0 / numpy.pi) * sa
    # sa_deg2 = (180.0 / numpy.pi) * sa2
    # print (a_deg,sa_deg,sa_deg2)
    return sa_deg

### Azimithal anlge and solid angle 

In [ ]:
def find_angle( p, c, d):
    pc,p = p
    v1 = p[1] - pc
    v2 = p[2] - pc
    nv = numpy.cross(v1, v2)
    cv = c - pc
    nnv = nv / numpy.linalg.norm(nv)
    ncv = cv / numpy.linalg.norm(cv)
    dp = abs(numpy.dot(nnv, ncv))
    ang = numpy.arccos(dp)
    ang_deg = (180 / numpy.pi) * ang
    s_ang = solid_angle(ang_deg, d)
    return ang_deg, s_ang

The following function extracts geometric information about aromatic residues from the pdb data

In [ ]:
def get_aromatic_info(pdb_data):
    aromatic_atoms = {
        'PHE': ['CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ', 'HD1', 'HD2', 'HE1', 'HE2', 'HZ'],
        'TYR': ['CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ', 'HD1', 'HD2', 'HE1', 'HE2', 'HH'],
        'TRP': ['CD2', 'CE2', 'CE3', 'CZ2', 'CZ3', 'CH2', 'HE3', 'HZ2', 'HZ3', 'HH2', 'HE1'],
        'HIS': ['CG', 'ND1', 'CD2', 'CE1', 'NE2', 'HD1', 'HD2', 'HE1', 'HE2', 'xx', 'yy']  # if needed un comment
    }
    ring_atoms = {
        'PHE': ['CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ'],
        'TYR': ['CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ'],
        'TRP': ['CD2', 'CE2', 'CE3', 'CZ2', 'CZ3', 'CH2'],
        'HIS': ['CG', 'ND1', 'CD2', 'CE1', 'NE2', ]  # if needed un comment
    }
    aromtic_residues = sorted(
        list(set([(int(i[0]), i[1], i[2]) for i in pdb_data[1].keys() if i[2] in aromatic_atoms.keys()])))
    ar_info = {}
    for m in pdb_data.keys():
        ar_info[m] = {}
        for ar_res in aromtic_residues:
            p = []
            for atm in ring_atoms[ar_res[2]]:
                p.append(pdb_data[m][(str(ar_res[0]), ar_res[1], ar_res[2], atm)])
            c = get_centroid(p)
            ar_info[m][(str(ar_res[0]), ar_res[1], ar_res[2])] = (c, p)
    return ar_info


In [ ]:
ar_info = get_aromatic_info(pdb_data)

In [ ]:
ar_info

The following function analyzes the NMR ensemble and calculates the mean, median, standard deviation, min and miximum of distance, azimithal angle and solid angle of the nearest aromatic ring. 

In [ ]:

def analyze_enzemble(ar_info,pdb_data):
    amide_res = {}
    for m in pdb_data.keys():
        amide_res[m]={}
        for atm in pdb_data[m].keys():
            if atm[3]=='H':
                amide_res[m][atm]=pdb_data[m][atm]
    dist={}
    angle={}
    solid_angle={}
    for atm in amide_res[1].keys():
        dist[atm]={}
        angle[atm]={}
        solid_angle[atm]={}
        for ar_res in ar_info[1].keys():
            d=[]
            a=[]
            sa=[]
            for m in amide_res.keys():
                d1=get_distance(ar_info[m][ar_res][0],amide_res[m][atm])
                d.append(d1)
                ang = find_angle(ar_info[m][ar_res],amide_res[m][atm],d1)
                a.append(ang[0])
                sa.append(ang[1])
            dist[atm][ar_res]=[round(numpy.mean(d),3),round(numpy.median(d),3),round(numpy.std(d),3),round(min(d),3),round(max(d),3)]
            angle[atm][ar_res]=[round(numpy.mean(a),3),round(numpy.median(a),3),round(numpy.std(a),3),round(min(a),3),round(max(a),3)]
            solid_angle[atm][ar_res]=[round(numpy.mean(sa),3),round(numpy.median(sa),3),round(numpy.std(sa),3),round(min(sa),3),round(max(sa),3)]
    s_dist={}
    for aa in dist.keys():
        sorted_dist = sorted(dist[aa].items(), key=lambda kv: kv[1][0])
        s_dist[aa]=sorted_dist
    return s_dist,angle,solid_angle


In [ ]:
sorted_dist,azi_angle,sold_angle=analyze_enzemble(ar_info,pdb_data)

All the data in dictionary format and we try to maintain the same key throuout the analysis

In [ ]:
sorted_dist

In [ ]:
azi_angle

In [ ]:
sold_angle

## Combine the above steps and analyze the amide-aromatic interaction for a given pair of PDB and BMRB ids
This is the final step in which we combine the information from two different database using atom identifier. As mentioned before, both data dictionaries are having same set of keys. First we find out which sequence numbering matches between the two files. Then information from both databases combined using matching sequence and atom identifier (dictionary keys) 

In [ ]:

def calculate_interaction(pdb,bmrb):
    pdb_data_actual = get_pdb_data(pdb)
    bmrb_data_actual = get_bmrb_data(bmrb)
    pdb_data_auth = get_pdb_data(pdb)
    bmrb_data_auth = get_bmrb_data(bmrb)
    pdb_actual_keys = [i for i in pdb_data_actual[1].keys() if i[3] == 'H']
    pdb_auth_keys = [i for i in pdb_data_auth[1].keys() if i[3] == 'H']
    bmrb_actual_keys = [i for i in bmrb_data_actual[0].keys() if i[3]=='H']
    bmrb_auth_keys = [i for i in bmrb_data_auth[0].keys() if i[3] == 'H']
    actual_actual = [i for i in bmrb_actual_keys if i in pdb_actual_keys]
    actual_auth = [i for i in bmrb_actual_keys if i in pdb_auth_keys]
    auth_actual = [i for i in bmrb_auth_keys if i in pdb_actual_keys]
    auth_auth= [i for i in bmrb_auth_keys if i in pdb_auth_keys]
    if len(bmrb_actual_keys)>0:
        seq_len = float(len(bmrb_actual_keys))
        actual_actual_match = float(len(actual_actual))/seq_len
        actual_auth_match = float(len(actual_auth))/seq_len
        auth_actual_match = float(len(auth_actual))/seq_len
        auth_auth_match = float(len(auth_auth))/seq_len
        if actual_actual_match > 0.7:
            pdb_data = pdb_data_actual
            bmrb_data = bmrb_data_actual
            tag_match = 'ORIG_ORIG'
        elif actual_auth_match > 0.7:
            pdb_data = pdb_data_auth
            bmrb_data = bmrb_data_actual
            tag_match = 'ORIG_AUTH'
        elif auth_actual_match > 0.7:
            pdb_data = pdb_data_actual
            bmrb_data = bmrb_data_auth
            tag_match = 'AUTH_ORIG'
        elif auth_auth_match > 0.7:
            pdb_data = pdb_data_auth
            bmrb_data = bmrb_data_auth
            tag_mathc = 'AUTH_AUTH'
        else:
            raise KeyError
        amide_chemical_shift, aromatic_chemical_shift, entity_size, assembly_size = bmrb_data
        ar_info = get_aromatic_info(pdb_data)
        dist,angle,solid_angle = analyze_enzemble(ar_info,pdb_data)
        if not os.path.isdir('./data'):
            os.system('mkdir ./data')
        if not os.path.isdir('./data/output'):
            os.system('mkdir ./data/output')
        fo_name = 'data/output/{}-{}-{}.csv'.format(pdb,bmrb,tag_match)
        fo=open(fo_name,'w')
        header = 'pdb_id,bmrb_id,entity_size,assembly_size,' \
                 'amide_seq_id,amide_chain_id,amide_res,amide_cs,amide_z,' \
                 'aro_seq_id,aro_chain_id,aro_res,' \
                 'mean_d,meidan_d,std_d,min_d,max_d,' \
                 'mena_ang,median_ang,std_and,min_ang,max_ang,' \
                 'mean_sang,median_sang,std_sang,min_sang,max_sang\n'
        fo.write(header)
        for atm in amide_chemical_shift.keys():
            # print (atm,amide_chemical_shift[atm],dist[atm][0],angle[atm][dist[atm][0][0]],
            #        solid_angle[atm][dist[atm][0][0]] )
            fo.write('{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(pdb,bmrb,
                                                                             entity_size,assembly_size,
                                                                             atm[0],atm[1],atm[2],
                   amide_chemical_shift[atm][0],amide_chemical_shift[atm][1],
                   dist[atm][0][0][0],dist[atm][0][0][1],dist[atm][0][0][2],
                   ','.join([str(i) for i in dist[atm][0][1]]),
                   ','.join([str(i) for i in angle[atm][dist[atm][0][0]]]),
                   ','.join([str(i) for i in solid_angle[atm][dist[atm][0][0]]])))
        fo.close()

### Example : Alalysis of 1WYO and 11086

In [ ]:
calculate_interaction('1WYO','11086')

In [ ]:
cat data/output/1WYO-11086-ORIG_ORIG.csv

## Plotting the results
Here is some of the example plots from the csv file

In [ ]:
def plot_d_vs_z(csv_file):
    with open(csv_file, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        z=[]
        d=[]
        ang=[]
        sang=[]
        lab=[]
        ar_res=[]
        for row in csvreader:
            info = '{}-{}-{}-{}-{}-{}/{}/{}/{}'.format(row[0],row[1],row[5],row[6],row[9],row[11],row[7],row[8],row[12])
            d.append(float(row[12]))
            z.append(float(row[8]))
            ang.append(float(row[17]))
            sang.append(float(row[22]))
            ar_res.append(row[11])
            lab.append(info)
    fig = px.scatter(x=d,y=z,color=ar_res,hover_name=lab,
                     labels=dict(x='Distance (Å)', y='Z-score'),)
    fig.show()

def plot_azimuthal_vs_z(csv_file):
    with open(csv_file, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        z = []
        d = []
        ang = []
        sang = []
        lab = []
        ar_res = []
        for row in csvreader:
            info = '{}-{}-{}-{}-{}-{}/{}/{}/{}'.format(row[0], row[1], row[5], row[6], row[9], row[11], row[7],
                                                       row[8], row[12])
            d.append(float(row[12]))
            z.append(float(row[8]))
            ang.append(float(row[17]))
            sang.append(float(row[22]))
            ar_res.append(row[11])
            lab.append(info)
    fig = px.scatter(x=ang, y=z, color=ar_res, hover_name=lab,
                     labels=dict(x='Azimuthal angle', y='Z-score'), )
    fig.show()

def plot_solidangle_vs_z(csv_file):
    with open(csv_file, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        z = []
        d = []
        ang = []
        sang = []
        lab = []
        ar_res = []
        for row in csvreader:
            info = '{}-{}-{}-{}-{}-{}/{}/{}/{}'.format(row[0], row[1], row[5], row[6], row[9], row[11], row[7],
                                                       row[8], row[12])
            d.append(float(row[12]))
            z.append(float(row[8]))
            ang.append(float(row[17]))
            sang.append(float(row[22]))
            ar_res.append(row[11])
            lab.append(info)
    fig = px.scatter(x=sang, y=z, color=ar_res, hover_name=lab,
                     labels=dict(x='Solid angle', y='Z-score'), )
    fig.show()
def plot_d_vs_solidangle(csv_file):
    with open(csv_file, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        z=[]
        d=[]
        ang=[]
        sang=[]
        lab=[]
        ar_res=[]
        for row in csvreader:
            info = '{}-{}-{}-{}-{}-{}/{}/{}/{}'.format(row[0],row[1],row[5],row[6],row[9],row[11],row[7],row[8],row[12])
            d.append(float(row[12]))
            z.append(float(row[8]))
            ang.append(float(row[17]))
            sang.append(float(row[22]))
            ar_res.append(row[11])
            lab.append(info)
    fig = px.scatter(x=d,y=sang,color=ar_res,hover_name=lab,
                     labels=dict(x='Distance (Å)', y='Solid angle'),)
    fig.show()

def plot_3d(csv_file):
    with open(csv_file, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        z = []
        d = []
        ang = []
        sang = []
        lab = []
        ar_res = []
        for row in csvreader:
            info = '{}-{}-{}-{}-{}-{}/{}/{}/{}'.format(row[0], row[1], row[5], row[6], row[9], row[11], row[7],
                                                       row[8], row[12])
            d.append(float(row[12]))
            z.append(float(row[8]))
            ang.append(float(row[17]))
            sang.append(float(row[22]))
            ar_res.append(row[11])
            lab.append(info)
    fig = px.scatter_3d(x=d, y=ang, z=z, color=ar_res, hover_name=lab,
                     labels=dict(x='Distance (Å)', z='Z-score', y='Azimuthal angle'), )
    fig.show()


In [ ]:
plot_d_vs_solidangle('data/output/1WYO-11086-ORIG_ORIG.csv')

In [ ]:
plot_d_vs_z('data/output/1WYO-11086-ORIG_ORIG.csv')

In [ ]:
plot_azimuthal_vs_z('data/output/1WYO-11086-ORIG_ORIG.csv')

In [ ]:
plot_solidangle_vs_z('data/output/1WYO-11086-ORIG_ORIG.csv')

In [ ]:
plot_3d('data/output/1WYO-11086-ORIG_ORIG.csv')

## Database wide analysis
We can generate the data for all matching PDB and BMRB entires and concatenate all the csv files. The above plotting functions can be used to analyze the concatenated csv file. 